In [2]:
#imports
import spacy
from spacy.tokens import Doc, Token
from spacy.matcher import Matcher
from typing import List, Tuple, Dict
import re
from lxml import etree
from bs4 import BeautifulSoup
import os
import pandas as pd

c:\Users\sgdbareh\AppData\Local\miniconda3\envs\EPO\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
#remove HTML tags
def RemoveHTMLTags(string):
     
    # Print string after removing tags
    x = re.compile(r'<[^>]+>').sub(' ', string)

    return x

In [5]:
def DocLength(text):

    return len(text)

In [6]:
def SaveBOW(df,name):

    df.to_csv(f'{name}', index=False)

In [18]:
def PreProcessing(df,nlp):
    
    #filter only english and exclude pdf links to orig docs
    df = df[(df[4]=='en') & (df[5]!='PDFEP')]
    df = df[[1,2,3,5,7]]
    #convert col to string
    df[7] = df[7].astype(str)
    #remove html tags
    df['nlp'] = df[7].apply(RemoveHTMLTags)
    #create spacy doc
    df["nlp"] = [doc for doc in nlp.pipe(df['nlp'].tolist())]
    #lowercase and alpha/ punctuation/ removes individual letters such as 'f' present in original text due to abbrev
    df['nlp'] = [[tok.lower_ for tok in doc if (tok.is_alpha) and (len(tok)>1) or (tok.lower_ in ['a','i'])] for doc in df['nlp']]

    return df
       

In [44]:
nlp = spacy.blank('en')

docLength = []
groups = []
uniq_patents = []

with open(os.path.join('C:\EPO Project\EPO Patent Data','test.txt'),'r',encoding='utf8') as f:
   
    df = pd.read_table(f,header=None)
    df = PreProcessing(df,nlp)
    df['doc_length'] = df['nlp'].apply(DocLength)
    group = df.groupby(5).size()
    groups.append(group)
    uniq_patents.append(len(df[1].unique()))
    SaveBOW(df,'file')


In [47]:
print("Total Tokens = ", df['doc_length'].sum())
print("Total Unique Patents = ", sum(uniq_patents))

Total Tokens =  3176
Total Unique Patents =  7


In [49]:
a = groups[0].copy()

In [50]:
groups.append(a)

In [59]:
#CONCAT
groups[0].merge(groups[1],how='left')

AttributeError: 'Series' object has no attribute 'merge'